**MARKET BASKET ANALYSIS NOTEBOOK**

Dowload and preprocess dataset

In [23]:
!pip install kaggle
!pip install pandas

import os,zipfile,json,re
import functools as ft

os.environ['KAGGLE_USERNAME']='giacomoturati1'
os.environ['KAGGLE_KEY']='7d34a1aefc3558065164b70c24ce27ed'

from kaggle.api.kaggle_api_extended import KaggleApi

def get_dataset():

  #execute only if the dataset was not already downloaded
  if 'old-newspaper.tsv' not in os.listdir():
    api=KaggleApi()
    api.authenticate()

    api.dataset_download_file('alvations/old-newspapers','old-newspaper.tsv')

    with zipfile.ZipFile('old-newspaper.tsv.zip','r') as _zip:
      _zip.extractall()

#languages: subset of languages to be considered during the market-basket analysis
def read_dataset_from_disk(languages=None):
  with open('old-newspaper.tsv','r') as f:
    #skip header line
    next(f)
    for line in f:
      l=line.split('\t')
      if languages is not None and l[0] not in languages: continue

      #get a list of words as basket skipping any sequence of non-alphabetical characters
      basket=re.split(r'[^a-zA-Z]+',l[3])
      #remove any empty string
      basket=[word.lower() for word in basket if word!='']
      yield basket

#create json wich contains an array of baskets (lists of words)
def create_test_json_dataset(languages=None):
  baskets=[]

  filename=ft.reduce(lambda i,j:i+'_'+j,languages) if languages is not None else "all_languages"
  filename+='.json'

  #execute only if the dataset was not already created
  if filename not in os.listdir():
    for line in read_dataset_from_disk(languages):
      baskets.append(line)

    f=open(filename,'w')
    f.write(json.dumps(baskets,indent="\t"))
    f.close()

#yield lists of words (baskets) from json file created with  
def iter_baskets_from_json(filename):
  theres_next=True
  basket=[]

  with open(filename,'r') as f:
    #skip first square braket
    next(f)
    for line in f:

      m=re.search(r'[a-zA-Z]+|\[|\]',line)
      line=line[m.start():m.end()]

      if line=="[": 
        basket=[]
        theres_next=True

      elif line=="]":
        if theres_next:
          theres_next=False
          yield basket

      else: basket.append(line)
    


get_dataset()
create_test_json_dataset(['Italian'])




In [24]:
baskets=iter_baskets_from_json('Italian.json')

print(next(baskets))

['con', 'l', 'editore', 'crocetti', 'il', 'poeta', 'e', 'drammaturgo', 'wole', 'soyinka', 'premio', 'nobel', 'per', 'la', 'letteratura', 'aprir', 'la', 'milanesiana', 'domenica', 'prossima', 'alle', 'presso', 'la', 'sala', 'buzzati', 'di', 'via', 'balzan', 'milano', 'con', 'lui', 'nacer', 'khemir', 'tahar', 'ben', 'jelloun', 'biyi', 'bandele', 'jean', 'hatzfeld', 'ben', 'okri', 'leggeranno', 'testi', 'sul', 'tema', 'bugie', 'e', 'verit', 'dello', 'stesso', 'argomento', 'parler', 'guido', 'barbujani', 'il', 'giugno', 'alle', 'con', 'un', 'intervento', 'che', 'pubblichiamo', 'a', 'pagina']


In [50]:
print(next(baskets))

['non', 'solo', 'con', 'la', 'nuova', 'pillola', 'si', 'semplifica', 'di', 'molto', 'la', 'cura', 'basta', 'prenderne', 'una', 'al', 'mattino', 'e', 'si', 'facilita', 'l', 'aderenza', 'stessa', 'alle', 'cure', 'non', 'alta', 'con', 'le', 'terapie', 'oggi', 'in', 'uso', 'che', 'richiedono', 'tutte', 'una', 'o', 'pi', 'iniezioni', 'al', 'd']
